# e2e

> Fill in a module description here

In [ ]:
#| hide
# from nbdev.showdoc import *

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import io
import json
import math
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorboard.plugins import projector


#from fastai.vision.all import *
#from portfolio_search.embeddings import *
#from portfolio_search.fileorg import *

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Loading TSV's and Centroids

In [ ]:
subclusters_dir = (
    "/Users/al/Dropbox/2-Areas/2-Sketchbooks/datasets/full-v2/04_SUBCLUSTERS"
)

In [ ]:
drawing_centroids_fname = f"{subclusters_dir}/centroids_drawings.json"
with open(drawing_centroids_fname, "r") as infile:
    drawing_centroids = json.load(infile)

In [ ]:
watercolor_centroids_fname = f"{subclusters_dir}/centroids_watercolors.json"
with open(watercolor_centroids_fname, "r") as infile:
    watercolor_centroids = json.load(infile)

In [ ]:
clustered_drawings_tsv_fname = f"{subclusters_dir}/clustered_drawings.tsv"
# drawings_df.to_csv(
#     clustered_drawings_tsv_fname, index=True, index_label="idx", sep="\t", header=True
# )
clustered_drawings_df = pd.read_csv(
    clustered_drawings_tsv_fname, delimiter="\t", index_col="idx"
)
clustered_drawings_df.head()

,abs_fname,rel_fname,label,pred_label,pred_idx,pred_probs,emb_csv,cluster,cluster_dist,metacluster,...,drawings_cluster48_id,drawings_cluster48_dist,drawings_cropped_cluster16_id,drawings_cropped_cluster16_dist,drawings_cropped_cluster32_id,drawings_cropped_cluster32_dist,drawings_cropped_cluster48_id,drawings_cropped_cluster48_dist,drawings_cropped_cluster64_id,drawings_cropped_cluster64_dist
idx,,,,,,,,,,,,,,,,,,,,,
0,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,art/sb77p043.jpg,art,xtra,4,"0.168532,0.008473,0.008068,0.016940,0.797986","-0.6198182,-0.5714155,-0.6871516,-0.6553513,-0...",11,199.22961,1,...,15,166.33705,15,265.844100,15,257.350560,15,268.67297,58,276.82034
1,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,art/sb67p021.jpg,art,xtra,4,"0.170976,0.008673,0.008480,0.017028,0.794843","1.1073987,0.3512016,-0.73929733,-0.6553513,1.7...",3,132.26111,0,...,45,137.20117,14,121.146545,31,122.172424,31,116.40921,34,122.04242
2,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,art/sb25p227.jpg,art,xtra,4,"0.158396,0.008277,0.007424,0.016584,0.809319","1.3776337,-0.5714155,-0.2587232,-0.6553513,1.3...",8,188.58075,0,...,8,148.63470,10,220.298950,1,217.335360,1,213.25168,1,191.67163
3,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,art/sb48p057.jpg,art,xtra,4,"0.165322,0.008453,0.008010,0.016587,0.801628","2.4936543,0.17563903,-0.6522293,-0.55642915,1....",14,90.26291,0,...,28,83.96362,2,144.254030,21,129.951970,21,140.07254,21,144.77570
4,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,art/sb77p044.jpg,art,xtra,4,"0.171638,0.008182,0.008061,0.016469,0.795650","2.170152,-0.01916939,-0.47766963,-0.6553513,-0...",4,176.29102,0,...,39,169.47894,15,151.277310,15,141.268980,15,145.39337,58,151.13809


In [ ]:
clustered_watercolors_tsv_fname = f"{subclusters_dir}/clustered_watercolors.tsv"
# watercolors_df.to_csv(
#     clustered_watercolors_tsv_fname,
#     index=True,
#     index_label="idx",
#     sep="\t",
#     header=True,
# )
clustered_watercolors_df = pd.read_csv(
    clustered_watercolors_tsv_fname, delimiter="\t", index_col="idx"
)
clustered_watercolors_df.head()

,abs_fname,rel_fname,label,pred_label,pred_idx,pred_probs,emb_csv,cluster,cluster_dist,metacluster,...,handlabeled_metacluster_name,handlabeled_metacluster_id,handlabeled_metacluster_was_correct,preprocessed_abs_path,handlabeled_abs_path,emb_csv_cropped,watercolors_cluster_id,watercolors_cluster_dist,watercolors_cropped_cluster_id,watercolors_cropped_cluster_dist
idx,,,,,,,,,,,,,,,,,,,,,
0,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,art/sb07p104.jpg,art,xtra,4,"0.177406,0.008799,0.008245,0.017177,0.788373","-0.39105213,0.96309596,-0.73929733,0.4489988,0...",5,238.33170,1,...,1_watercolors,1,True,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,"-0.6198182,0.8895926,-0.73929733,0.6418346,0.4...",7,215.02927,7,206.64117
1,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,art/sb31p017.jpg,art,xtra,4,"0.172119,0.008924,0.008226,0.016950,0.793781","-0.05104202,-0.3935787,-0.73929733,0.86878717,...",11,191.66959,1,...,1_watercolors,1,True,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,"1.491735,-0.5714155,-0.73929733,0.05867392,0.5...",4,199.03098,4,195.97546
2,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,art/sb05p079.jpg,art,xtra,4,"0.172158,0.008873,0.008536,0.017017,0.793416","1.8089037,-0.5714155,-0.73929733,-0.10773122,-...",11,140.32997,1,...,1_watercolors,1,NaN,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,"-0.6013173,-0.5714155,-0.7368871,-0.6553513,-0...",4,142.76602,4,248.37231
3,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,art/sb26p082.jpg,art,xtra,4,"0.179525,0.008828,0.008282,0.017196,0.786169","1.8392665,0.3796271,-0.73929733,0.82189727,1.6...",7,176.11029,1,...,1_watercolors,1,NaN,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,"0.8175719,0.48385304,-0.73929733,1.0291914,1.4...",1,157.93832,1,196.84943
4,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,art/sb69p101-color.jpg,art,xtra,4,"0.179525,0.008828,0.008282,0.017196,0.786169","-0.23259398,-0.5714155,-0.73929733,-0.6553513,...",7,187.45346,1,...,1_watercolors,1,NaN,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,/Users/al/Dropbox/2-Areas/2-Sketchbooks/datase...,"0.14908737,-0.5714155,-0.73929733,0.44245207,-...",1,167.27823,1,136.63800


In [ ]:
%load_ext tensorboard
import csv

import numpy as np
import tensorflow as tf
from PIL import Image

In [ ]:
drawings_emb = np.stack(
    [
        np.array([float(f) for f in s.split(",")])
        for s in list(clustered_drawings_df.emb_csv)
    ]
).astype(np.float32)

drawings_cropped_emb = np.stack(
    [
        np.array([float(f) for f in s.split(",")])
        for s in list(clustered_drawings_df.emb_csv_cropped)
    ]
).astype(np.float32)

drawings_emb.shape, drawings_cropped_emb.shape

((1923, 512), (1923, 512))

In [ ]:
watercolors_emb = np.stack(
    [
        np.array([float(f) for f in s.split(",")])
        for s in list(clustered_watercolors_df.emb_csv)
    ]
).astype(np.float32)

watercolors_cropped_emb = np.stack(
    [
        np.array([float(f) for f in s.split(",")])
        for s in list(clustered_watercolors_df.emb_csv_cropped)
    ]
).astype(np.float32)

watercolors_emb.shape, watercolors_cropped_emb.shape

((404, 512), (404, 512))

In [ ]:
weights = tf.Variable(drawings_emb)

Metal device set to: Apple M1 Pro


In [ ]:
weights

<tf.Variable 'Variable:0' shape=(1923, 512) dtype=float32, numpy=
array([[-0.6198182 , -0.5714155 , -0.6871516 , ...,  0.09847462,
        -0.49206764, -0.39251342],
       [ 1.1073987 ,  0.3512016 , -0.73929733, ..., -0.65107447,
        -0.49206764,  0.32716987],
       [ 1.3776337 , -0.5714155 , -0.2587232 , ..., -0.65107447,
        -0.4748875 ,  0.19909522],
       ...,
       [ 1.637738  , -0.5714155 , -0.73929733, ..., -0.65107447,
        -0.13707963,  0.20361403],
       [ 4.6632442 ,  0.29762375, -0.73929733, ...,  0.01897907,
        -0.49206764, -0.39251342],
       [ 0.71924925,  0.20198071,  1.2284024 , ..., -0.22790754,
         0.7221783 , -0.08996937]], dtype=float32)>

In [ ]:
len(clustered_drawings_df)

1923

In [ ]:
tf_logs = f"{subclusters_dir}/tflogs"
if not os.path.isdir(tf_logs):
    os.makedirs(tf_logs)

tf_embeddings = f"{tf_logs}/embeddings/"
if not os.path.isdir(tf_embeddings):
    os.makedirs(tf_embeddings)

In [ ]:
metadata_fname = f"{tf_embeddings}/metadata.tsv"

with open(metadata_fname, 'w') as outfile:
    for idx in range(len(clustered_drawings_df)):
        row = clustered_drawings_df.iloc[idx]
        outfile.write(f"{row.indiv_fname}\n")

!head {metadata_fname}

sb77p043.jpg
sb67p021.jpg
sb25p227.jpg
sb48p057.jpg
sb77p044.jpg
sb11p024.jpg
sb67p006--kitchen-nap.jpg
sb68p093.jpg
sb45p121.jpg
sb69p013.jpg


In [ ]:
# images_pil = []

# for idx in range(len(clustered_drawings_df)):
#     row = clustered_drawings_df.iloc[idx]
#     # if idx > 10:
#     #     break
#     images_pil.append(Image.open(row.preprocessed_abs_path))

In [ ]:
one_square_size = int(np.ceil(np.sqrt(len(clustered_drawings_df))))
one_square_size

44

In [ ]:
from PIL import Image

master_width = 100 * one_square_size
master_height = 100 * one_square_size
spriteimage = Image.new(
    mode='RGBA',
    size=(master_width, master_height),
    color=(0,0,0,0) # fully transparent
)
for idx in range(len(clustered_drawings_df)):
    row = clustered_drawings_df.iloc[idx]
    image = Image.open(row.preprocessed_abs_path).resize((100, 100))
    div, mod = divmod(idx, one_square_size)
    h_loc = 100 * div
    w_loc = 100 * mod
    spriteimage.paste(image, (w_loc, h_loc))
spriteimage.convert('RGB').save(f'{tf_embeddings}/sprite.jpg', transparency=0)


In [ ]:
import csv

vecs_fname = f"{tf_embeddings}/feature_vecs.tsv"

with open(vecs_fname, 'w') as outfile:
    csv_writer = csv.writer(outfile, delimiter="\t")
    csv_writer.writerows([e.split(',') for e in list(clustered_drawings_df.emb_csv)])

!head -n3 {vecs_fname}

-0.6198182	-0.5714155	-0.6871516	-0.6553513	-0.6254566	-0.65341413	-0.7267234	-0.4896673	-0.7537334	-0.42715576	1.4904574	-0.72562873	-0.662768	-0.6266761	0.6913696	0.054546177	-0.5467274	-0.6520286	0.117179275	0.94330966	0.5815062	-0.55193967	-0.5446661	-0.6779002	-0.57759243	-0.4584825	-0.3818994	-0.5750927	-0.73986673	-0.45211998	-0.6141643	-0.67007643	-0.52171326	-0.4592305	1.4788612	-0.6186755	0.799198	-0.57338566	-0.17778617	0.10336053	0.093812525	0.17924881	-0.6189488	-0.6682474	2.1040225	-0.59257406	1.6553459	-0.6001429	0.5109105	0.19352531	-0.2517513	-0.66552746	-0.6482407	-0.4732165	-0.7311095	-0.6058329	-0.41851217	-0.6646769	0.19708657	-0.6859715	0.547243	-0.47478092	-0.8143768	0.6043384	-0.45561737	-0.5205148	-0.19637269	-0.67944384	-0.5505594	-0.7467994	-0.64460987	-0.46079588	-0.69849855	-0.41446227	-0.6081119	-0.6608871	0.23214674	1.7035191	-0.47101474	-0.5426106	-0.5526179	1.4402422	0.37750077	-0.4790987	0.46338224	-0.03557557	-0.577085	-0.6104121	0.31350982	0.63637584

In [ ]:
tf_embeddings

'/Users/al/Dropbox/2-Areas/2-Sketchbooks/datasets/full-v2/04_SUBCLUSTERS/tflogs/embeddings/'

```         
<tf.Variable 'Variable:0' shape=(1923, 512) dtype=float32, numpy=
array([[-0.6198182 , -0.5714155 , -0.6871516 , ...,  0.09847462,
        -0.49206764, -0.39251342],
       [ 1.1073987 ,  0.3512016 , -0.73929733, ..., -0.65107447,
        -0.49206764,  0.32716987],
       [ 1.3776337 , -0.5714155 , -0.2587232 , ..., -0.65107447,
        -0.4748875 ,  0.19909522],
       ...,
       [ 1.637738  , -0.5714155 , -0.73929733, ..., -0.65107447,
        -0.13707963,  0.20361403],
       [ 4.6632442 ,  0.29762375, -0.73929733, ...,  0.01897907,
        -0.49206764, -0.39251342],
       [ 0.71924925,  0.20198071,  1.2284024 , ..., -0.22790754,
         0.7221783 , -0.08996937]], dtype=float32)>
```

In [ ]:
weights

# weights = tf.Variable(model.layers[0].get_weights()[0][1:])

<tf.Variable 'Variable:0' shape=(1923, 512) dtype=float32, numpy=
array([[-0.6198182 , -0.5714155 , -0.6871516 , ...,  0.09847462,
        -0.49206764, -0.39251342],
       [ 1.1073987 ,  0.3512016 , -0.73929733, ..., -0.65107447,
        -0.49206764,  0.32716987],
       [ 1.3776337 , -0.5714155 , -0.2587232 , ..., -0.65107447,
        -0.4748875 ,  0.19909522],
       ...,
       [ 1.637738  , -0.5714155 , -0.73929733, ..., -0.65107447,
        -0.13707963,  0.20361403],
       [ 4.6632442 ,  0.29762375, -0.73929733, ...,  0.01897907,
        -0.49206764, -0.39251342],
       [ 0.71924925,  0.20198071,  1.2284024 , ..., -0.22790754,
         0.7221783 , -0.08996937]], dtype=float32)>

In [ ]:

# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(tf_embeddings, "embedding.ckpt"))

checkpoint.save(os.path.join(tf_logs, "embedding.ckpt"))

'/Users/al/Dropbox/2-Areas/2-Sketchbooks/datasets/full-v2/04_SUBCLUSTERS/tflogs/embedding.ckpt-2'

In [ ]:
# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/Variable:0"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(tf_embeddings, config)

In [ ]:

%tensorboard --logdir /Users/al/Dropbox/2-Areas/2-Sketchbooks/datasets/full-v2/04_SUBCLUSTERS/tflogs


Launching TensorBoard...

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()